In [61]:
import pandas as pd
import numpy as np
from utils import reduce_memory_usage
from utils import break_datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_percentage_error,mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import joblib
import zipfile
import os
import category_encoders
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

# Loading data

In [2]:
df = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/FilteredDataset/train.csv').drop('Unnamed: 0',axis=1)


In [3]:
df['log_meter_reading']=np.log1p(df['meter_reading'])
df['log_square_feet']=np.log1p(df['square_feet'])
df= break_datetime(df)
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,sea_level_pressure,wind_direction,wind_speed,log_meter_reading,log_square_feet,year,weekofyear,dayofweek,month,hour
0,46,0,2016-01-01,53.2397,0,Retail,9045,2016.0,NaN,25.0,...,1019.5,0.0,0.0,3.993413,9.110078,2015,53,5,1,0
1,74,0,2016-01-01,43.0013,0,Parking,387638,1997.0,NaN,25.0,...,1019.5,0.0,0.0,3.784219,12.867830,2015,53,5,1,0
2,93,0,2016-01-01,52.4206,0,Office,33370,1982.0,NaN,25.0,...,1019.5,0.0,0.0,3.978196,10.415443,2015,53,5,1,0
3,105,0,2016-01-01,23.3036,1,Education,50623,NaN,5.0,3.8,...,1021.0,240.0,3.1,3.190624,10.832181,2015,53,5,1,0
4,106,0,2016-01-01,0.3746,1,Education,5374,NaN,4.0,3.8,...,1021.0,240.0,3.1,0.318163,8.589514,2015,53,5,1,0


# Missing Value Imputation

In [4]:
def percent_missing_val(df):

  percent_missing = (df.isnull().sum() * 100) / len(df)
  missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
  return missing_value_df

In [5]:
missing_value_df= percent_missing_val(df)


In [6]:
missing_value_df

,column_name,percent_missing
building_id,building_id,0.000000
meter,meter,0.000000
timestamp,timestamp,0.000000
meter_reading,meter_reading,0.000000
site_id,site_id,0.000000
primary_use,primary_use,0.000000
square_feet,square_feet,0.000000
year_built,year_built,54.890488
floor_count,floor_count,74.735249
air_temperature,air_temperature,0.398257


As the year built and floor count columns have more than 50% values are missing, so we will drop these two columns.As the year built and floor count columns have more than 50% values are missing, so we will drop these two columns.

In [7]:
df.drop(['year_built', 'floor_count'], axis=1,inplace=True)


For the weather features, fill the missing values with the daily mean value of the corresponding feature in the site

In [9]:
def nan_fillers(df):
  air_temp_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['air_temperature'].transform('mean')
  df['air_temperature'].fillna(air_temp_df, inplace=True)

  dew_temp_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['dew_temperature'].transform('mean')
  df['dew_temperature'].fillna(dew_temp_df, inplace=True)

  cloud_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['cloud_coverage'].transform('mean')
  df['cloud_coverage'].fillna(cloud_df, inplace=True)

  sea_level_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['sea_level_pressure'].transform('mean')
  df['sea_level_pressure'].fillna(sea_level_df, inplace=True)

  precip_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['precip_depth_1_hr'].transform('mean')
  df['precip_depth_1_hr'].fillna(precip_df, inplace=True)

  wind_dir_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['wind_direction'].transform('mean')
  df['wind_direction'].fillna(wind_dir_df, inplace=True)

  wind_speed_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['wind_speed'].transform('mean')
  df['wind_speed'].fillna(wind_speed_df, inplace=True)


  return df

In [10]:
df= nan_fillers(df)

In [11]:
df.isnull().sum()

building_id                 0
meter                       0
timestamp                   0
meter_reading               0
site_id                     0
primary_use                 0
square_feet                 0
air_temperature             0
cloud_coverage         713295
dew_temperature             0
precip_depth_1_hr     2075909
sea_level_pressure     797538
wind_direction              0
wind_speed                  0
log_meter_reading           0
log_square_feet             0
year                        0
weekofyear                  0
dayofweek                   0
month                       0
hour                        0
dtype: int64

As there were lot of slices where all the values were NAN in 3 columns namely: cloud_coverage, precip_depth_1_hr and sea_level_pressure hence we will impute the rest of the nan values with the median value.

In [12]:
df['cloud_coverage'].fillna(df['cloud_coverage'].median(), inplace=True)
df['sea_level_pressure'].fillna(df['sea_level_pressure'].median(), inplace=True)
df['precip_depth_1_hr'].fillna(df['precip_depth_1_hr'].median(), inplace=True)

# Baseline Model: Decision Tree Regressor

For baseline model, we will use site id, primary use, square feet and air temperature as features and predict the consumption in hourly 
* The building id and site id are highly correlated, so we will use one of them
* The air temperature and dew temperature are highly correlated, so we will use one of them
* square feet has positive correalation with the meter readings, so we will use as a feature


### Train - Test Split

Split data so that first 8 months will be train set and last two months will be test set

In [121]:
df=df.sort_values(by='timestamp')
X_train, X_test= train_test_split(df, test_size=0.20, shuffle=False)

In [122]:
y_train = X_train['log_meter_reading']
y_test = X_test['log_meter_reading']
X_train.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)
X_test.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)

# Label Encoding

In [123]:
label_enc= LabelEncoder()
label_enc.fit(df['primary_use'])
X_train['primary_use']= label_enc.transform(X_train['primary_use'])
X_test['primary_use']= label_enc.transform(X_test['primary_use'])


In [124]:
X_train.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,log_square_feet,year,weekofyear,dayofweek,month,hour
0,46,0,2016-01-01,0,11,9045,25.000000,6.000000,20.000000,-0.138889,1019.500000,0.000000,0.000000,9.110078,2015,53,5,1,0
486,597,0,2016-01-01,4,9,189425,6.062753,0.957311,-4.979562,0.000000,1021.197439,85.513393,2.575774,12.151754,2015,53,5,1,0
495,608,0,2016-01-01,4,1,124197,6.062753,0.957311,-4.979562,0.000000,1021.197439,85.513393,2.575774,11.729632,2015,53,5,1,0
577,696,0,2016-01-01,5,0,19041,6.913043,0.000000,5.434783,0.000000,1016.500000,123.181818,8.017391,9.854402,2015,53,5,1,0
502,616,0,2016-01-01,4,0,117814,6.062753,0.957311,-4.979562,0.000000,1021.197439,85.513393,2.575774,11.676871,2015,53,5,1,0


In [125]:
features = ['site_id','air_temperature','log_square_feet','primary_use','hour','dayofweek','month']

In [126]:
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)

DecisionTreeRegressor(max_depth=13)

In [127]:
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

MSE for train set is:  0.25261274825351854
R2 for train set is:  0.8892271981704003
MAPE for train set is:  0.19947484161186446
MAE for train set is:  0.33862885553222727


In [128]:

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for train set is: ",test_error_mse)

print("R2 for train set is: ",test_error_r2)

print("MAPE for train set is: ",test_error_mape)

print("MAE for train set is: ",test_error_mae)


MSE for train set is:  0.3295410441218216
R2 for train set is:  0.8556875363568515
MAPE for train set is:  0.23986857811000253
MAE for train set is:  0.379661784081513


# Baseline Model: Linear Regression

### Train - Test Split

In [142]:
df=df.sort_values(by='timestamp')
X_train, X_test= train_test_split(df, test_size=0.20, shuffle=False)
y_train = X_train['log_meter_reading']
y_test = X_test['log_meter_reading']
X_train.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)
X_test.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)
X_train = X_train[features]
X_test = X_test[features]

### One Hot Encoding

In [143]:
categorical_features = ["site_id","primary_use"]

In [144]:
X_train = pd.get_dummies(X_train, columns=categorical_features)
X_test = pd.get_dummies(X_test, columns=categorical_features )

In [149]:
LR = LinearRegression()
LR.fit(X_train,y_train)

LinearRegression()

In [151]:
y_pred_train = LR.predict(X_train)
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

MSE for train set is:  0.7785728147304347
R2 for train set is:  0.6585882682815123
MAPE for train set is:  6.131150736106205
MAE for train set is:  0.6488163076662329


In [153]:
y_pred_test = LR.predict(X_test)
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for train set is: ",test_error_mse)

print("R2 for train set is: ",test_error_r2)

print("MAPE for train set is: ",test_error_mape)

print("MAE for train set is: ",test_error_mae)

MSE for train set is:  0.8133450164875552
R2 for train set is:  0.64382469999551
MAPE for train set is:  5.850426429813771
MAE for train set is:  0.659385587270123


# Save the models

 Save Decision Tree Regressor

In [157]:
# Specify the zip file name
zip_filename = "../models/DTR_v0.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "temp_model.pkl"
    joblib.dump(DTR, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="DTR_v0.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)


Save Linear Regression

In [158]:
# Specify the zip file name
zip_filename = "../models/LR_v0.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "temp_model.pkl"
    joblib.dump(DTR, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="LR_v0.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)

Reuse the model by loading

In [162]:
# Specify the ZIP file name
zip_filename = "../models/LR_v0.zip"

# Extract the model file from the ZIP archive
with zipfile.ZipFile(zip_filename, "r") as archive:
    # Extract the model file (named "your_model.pkl" in this example)
    archive.extract("LR_v0.pkl")
    
# Load the model
model = joblib.load("LR_v0.pkl")  # Replace with "pickle.load" if you used pickle

os.remove("LR_v0.pkl")

# You can now use the "model" for predictions or other tasks
y_pred_test = LR.predict(X_test)
test_error = mean_squared_error(y_test,y_pred_test)
print("MSE for test set is: ",test_error)

MSE for test set is:  0.8133450164875552
